Remember to update BEDTools:
```sh
conda install -c bioconda bedtools
```

In [1]:
import h5py
import scipy.sparse as sp
import numpy as np
import os
import pandas as pd
import pybedtools


In [2]:
os.chdir("/disk2/user/gabgam/the_project/5_integration_and_correlation/")
print(os.getcwd())

/disk2/user/gabgam/the_project/5_integration_and_correlation


In [9]:
# Load the file
file_path = "../data/spatial_atac/combined/counts_brca_satac.h5"
f = h5py.File(file_path, 'r')

In [11]:
temp = f['unknown']
temp.keys()

<KeysViewHDF5 ['barcodes', 'data', 'gene_names', 'genes', 'indices', 'indptr', 'shape']>

In [16]:
import collections
import scipy.sparse as sp_sparse
import tables

FeatureBCMatrix = collections.namedtuple('FeatureBCMatrix', ['ids', 'names', 'barcodes', 'matrix'])

def get_matrix_from_h5(filename):
    with tables.open_file(filename, 'r') as f:
        try:
            group = f.get_node(f.root, 'unknown')
        except tables.NoSuchNodeError:
            print("Matrix group does not exist in this file.")
            return None
        #feature_group = getattr(group, 'features').read()
        ids = getattr(group, 'genes').read()
        names = getattr(group, 'gene_names').read()
        barcodes = getattr(group, 'barcodes').read()
        data = getattr(group, 'data').read()
        indices = getattr(group, 'indices').read()
        indptr = getattr(group, 'indptr').read()
        shape = getattr(group, 'shape').read()
        matrix = sp_sparse.csc_matrix((data, indices, indptr), shape=shape)
        return FeatureBCMatrix(ids, names, barcodes, matrix)



tf_bc_matrix = get_matrix_from_h5(file_path)
tf_bc_matrix


FeatureBCMatrix(ids=array([b'chr1-975451-975952', b'chr1-1014228-1014729',
       b'chr1-1290080-1290581', ..., b'chrX-155612714-155613215',
       b'chrX-155767373-155767874', b'chrX-155881033-155881534'],
      dtype='|S25'), names=array([b'chr1-975451-975952', b'chr1-1014228-1014729',
       b'chr1-1290080-1290581', ..., b'chrX-155612714-155613215',
       b'chrX-155767373-155767874', b'chrX-155881033-155881534'],
      dtype='|S25'), barcodes=array([b'AAACAAGGGATCAAAT-1_1', b'AAACAGCAGTCTGCTA-1_1',
       b'AAACATTCGGGATTCT-1_1', ..., b'TTGTTGTGCCCTTGAA-1_3',
       b'TTGTTGTGGAGACAGT-1_3', b'TTGTTTCATTACGCTT-1_3'], dtype='|S20'), matrix=<215978x9866 sparse matrix of type '<class 'numpy.float64'>'
	with 11091248 stored elements in Compressed Sparse Column format>)

In [18]:
tf_bc_matrix.read()

AttributeError: 'FeatureBCMatrix' object has no attribute 'read'

In [10]:
list(f.keys())

['unknown']

In [52]:
temp.values()

ValuesViewHDF5(<HDF5 group "/unknown" (7 members)>)

In [13]:
f['unknown']['genes'][:]

array([b'chr1-975451-975952', b'chr1-1014228-1014729',
       b'chr1-1290080-1290581', ..., b'chrX-155612714-155613215',
       b'chrX-155767373-155767874', b'chrX-155881033-155881534'],
      dtype='|S25')

In [54]:
f['unknown']['shape'][:]

array([215978,   9866], dtype=int32)

In [55]:
with h5py.File(file_path, "r") as h5_file:
    barcodes = np.array(h5_file["unknown/barcodes"]).astype(str)
    peaks = np.array(h5_file["unknown/gene_names"]).astype(str)  # Actually chromatin regions
    data = np.array(h5_file["unknown/data"])
    indices = np.array(h5_file["unknown/indices"])
    indptr = np.array(h5_file["unknown/indptr"])
    shape = (len(barcodes), len(peaks))  # Correct shape order

counts_matrix = sp.csr_matrix((data, indices, indptr), shape=shape)


In [56]:
# Read gene annotations
refgene = pd.read_csv("utils/refGene.txt", sep="\t", header=None)
refgene = refgene[[2, 4, 5, 12]]  # Keep chr, start, end, gene_name
refgene.columns = ["chr", "start", "end", "gene_name"]
refgene["chr"] = "chr" + refgene["chr"].astype(str)  # Format chromosomes

In [57]:
peaks

array(['chr1-975451-975952', 'chr1-1014228-1014729',
       'chr1-1290080-1290581', ..., 'chrX-155612714-155613215',
       'chrX-155767373-155767874', 'chrX-155881033-155881534'],
      dtype='<U25')

In [58]:

# Convert ATAC peaks to BED format
peaks_df = pd.DataFrame([[p.split("-")[0], str(p.split("-")[1]+"-"+p.split("-")[2])] for p in peaks], columns=["chr", "pos"])
peaks_df[["start", "end"]] = peaks_df["pos"].str.split("-", expand=True)
peaks_df = peaks_df.drop(columns=["pos"])
peaks_df["start"] = peaks_df["start"].astype(int)
peaks_df["end"] = peaks_df["end"].astype(int)

# Convert to BedTool objects
peaks_bed = pybedtools.BedTool.from_dataframe(peaks_df).sort()
genes_bed = pybedtools.BedTool.from_dataframe(refgene).sort()


In [59]:
# Now find the nearest gene
closest_genes = peaks_bed.closest(genes_bed, d=True)
closest_genes_df = closest_genes.to_dataframe(
    names=["peak_chr", "peak_start", "peak_end", "gene_chr", "gene_start", "gene_end", "gene_name", "distance"]
)

closest_genes_df

,peak_chr,peak_start,peak_end,gene_chr,gene_start,gene_end,gene_name,distance
0,chr1,9872,10373,.,-1,-1,.,-1
1,chr1,17232,17733,.,-1,-1,.,-1
2,chr1,180632,181133,.,-1,-1,.,-1
3,chr1,181205,181706,.,-1,-1,.,-1
4,chr1,183555,184056,.,-1,-1,.,-1
...,...,...,...,...,...,...,...,...
215973,chrY,56870726,56871227,.,-1,-1,.,-1
215974,chrY,56871262,56871763,.,-1,-1,.,-1
215975,chrY,56872688,56873189,.,-1,-1,.,-1
215976,chrY,56873413,56873914,.,-1,-1,.,-1


In [60]:
# Keep only peaks within 10kb of a gene
closest_genes_df = closest_genes_df[closest_genes_df["distance"] <= 10000]

In [61]:
# Aggregate chromatin accessibility for each gene
import numpy as np

gene_activity = closest_genes_df.groupby("gene_name")["peak_start"].count()
gene_activity = gene_activity / gene_activity.max()  # Normalize


In [62]:
gene_activity.shape

(1,)